In [111]:
import time
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.sparse import csr_matrix, hstack
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer

import warnings
warnings.filterwarnings('ignore')

import gc
import csv

In [115]:
df_train = pd.read_csv('train.csv', quoting=csv.QUOTE_NONE, warn_bad_lines=False, error_bad_lines=False)

In [117]:
df_train.head()

,,,,,,PRODUCT_ID,TITLE,BULLET_POINTS,DESCRIPTION,PRODUCT_TYPE_ID,PRODUCT_LENGTH
1925202,"""ArtzFolio Tulip Flowers Blackout Curtain for Door","Window & Room | Eyelets & Tie Back | Canvas Fabric | Width 4.5feet (54inch) Height 5 feet (60 inch); Set of 2 PCS""","""[LUXURIOUS & APPEALING: Beautiful custom-made curtains to decorate any home or office | Includes inbuilt tieback to hold the curtain | Completely finished and ready to hang on walls & windows",MATERIAL: Luxurious & versatile fabric with a natural finish | High colour fastness | State-of-the-art digital printing ensures colour consistency and prevents any fading | Eyelets; Cotton Canvas; Width 4.5feet (54inch) | Multicolour | PACKAGE: 2 Room Curtains Eyelets | SIZE: Height 5 feet (60 inch); SET OF 2 PCS,BLACKOUT CURTAIN: 100% opaque & heavy premium cotton canvas fabric | Tight knitted,long life & durable fabric | Printing only on...,MADE TO PERFECTION: Large eyelets at the top t...,QUALITY ASSURED: Gentle wash with similar colo...,NaN,1650.0,2125.98
2673191,Marks & Spencer Girls' Pyjama Sets T86_2561C_Navy Mix_9-10Y,"""[Harry Potter Hedwig Pyjamas (6-16 Yrs)",100% cotton (exclusive of trimmings),Ribbing - 98% cotton,"2% elastane]""",NaN,2755,393.7,NaN,NaN,NaN
283658,The United Empire Loyalists: A Chronicle of the Great Migration,NaN,NaN,6112,598.424,NaN,NaN,NaN,NaN,NaN,NaN
413758,Ungifted: My Life and Journey,NaN,NaN,23,598,NaN,NaN,NaN,NaN,NaN,NaN
2050239,PUMA Cali Sport Clean Women's Sneakers White Leather (37540701),"""[Style Name:-Cali Sport Clean Women's Sneakers",Upper Material:-Leather,Lower Material:-Rubber,Care Instructions: Wipe with a clean dry cloth,"Warranty Period: 90 Days Manufacturers Warranty]""",NaN,3302,393.7,NaN,NaN


In [118]:
print("Size of the training dataset is", df_train.shape)

Size of the training dataset is (228926, 6)


In [119]:
df_train.dtypes

PRODUCT_ID         object
TITLE              object
BULLET_POINTS      object
DESCRIPTION        object
PRODUCT_TYPE_ID    object
PRODUCT_LENGTH     object
dtype: object

In [120]:
df_train.astype('object').describe().transpose()

,count,unique,top,freq
PRODUCT_ID,131163,73977,600,1640
TITLE,115161,69614,600,1085
BULLET_POINTS,104606,60322,600,630
DESCRIPTION,95309,51568,590.5511805,1530
PRODUCT_TYPE_ID,81101,44854,590.5511805,584
PRODUCT_LENGTH,67650,38486,12064,726


In [121]:
df_train.isna().sum()

PRODUCT_ID          97763
TITLE              113765
BULLET_POINTS      124320
DESCRIPTION        133617
PRODUCT_TYPE_ID    147825
PRODUCT_LENGTH     161276
dtype: int64

In [122]:
df_train=df_train.dropna()

In [123]:
df_train.isna().sum()

PRODUCT_ID         0
TITLE              0
BULLET_POINTS      0
DESCRIPTION        0
PRODUCT_TYPE_ID    0
PRODUCT_LENGTH     0
dtype: int64

In [125]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import sent_tokenize, word_tokenize

from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.model_selection import train_test_split

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [126]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer

In [127]:
lb = LabelBinarizer(sparse_output=True)
tv = TfidfVectorizer(max_features=55000, ngram_range=(1, 2), stop_words='english')

In [128]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [129]:
stop = stopwords.words('english')
lem = WordNetLemmatizer()

<b> BULLET POINTS </b>

In [130]:
def cleanText(words):
    """The function to clean text"""
    words = re.sub("[^a-zA-Z]"," ",words)
    text = words.lower().split()
    return " ".join(text)

df_train['BULLET_POINTS'] = df_train['BULLET_POINTS'].apply(cleanText)

In [131]:
def remove_stopwords(text):
    """The function to removing stopwords"""
    text = [word.lower() for word in text.split() if word.lower() not in stop]
    return " ".join(text)

In [132]:
def word_lem(text):
    """The function to apply lemmatizing"""
    lem_text = [lem.lemmatize(word) for word in text.split()]
    return " ".join(lem_text)
df_train['BULLET_POINTS'] = df_train['BULLET_POINTS'].apply(remove_stopwords)
df_train['BULLET_POINTS'] = df_train['BULLET_POINTS'].apply(word_lem)

In [134]:
df_train.head()

,,,,,,PRODUCT_ID,TITLE,BULLET_POINTS,DESCRIPTION,PRODUCT_TYPE_ID,PRODUCT_LENGTH
1329311,"""Amscan 354001.54 Jumbo Margarita Glass",30 oz,"Caribbean Blue""","""[30 oz.",High quality paper,Drink in style with this plastic margarita glass,Perfect for parties,celebration add classy simple accent table set...,Serve cool drinks in our 30 oz Jumbo Plastic M...,1419.0,950.0
798322,"""K-Swiss 201 Classic Tennis Shoe (Infant/Toddler)",Black/Black,"5 M US Toddler""","""[Sleek low-top sneaker with vertically stitched toe",D-ring hardware lacing,"Perforated tongue]""","""K-Swiss 201 Classic Tennis Shoe (Infant/Toddler)",black black,"5 M US Toddler""",2449.0,750.0
1629663,"""Liakada Girls Argyle Supportive Sports Bra with Scoop Neckline Dance",Gym,Yoga,"Cheer! Black""","""[This sports bra top features contract criss-cross straps on the back and a scoop neckline for a clean finish.",Integrated bra shelf liner,Adjustable straps in a woven pattern,nylon spandex,This sports bra top features contract criss-cr...,2640.0,800.0
1833730,Khadims Men Black Slip-On Shoe - UK 9,"""[Upper Material: Synthetic / Sole Material: Airmax",Occasion: Formal / Utility: Office Formals,Closure: Slip-on / Toe Style: Closed,Shoe Style: Slip-On / Shoe Type: Shoe,Care Instructions: 1. Use a soft shoe brush to...,"""Khadim’s combines comfort and durability with...",pair khadim foot give extra confidence take gr...,Khadim’s gives you an unmatched value-for-mon...,3246.0,1279.527558
2656446,TONYBOY Boy's Yarn Dyed Checks Shirt & Solid Shirt - Green/Fluo Orange (Pack of 2),"""[Fit Type: Regular Fit",Material: Cotton Fabric,Pattern : 1pcs. Y/D Checks Shirt || 1 pc. Solid Shirt,Sleeve: Full Sleeve || Neck: Regular Collar,"Wash Care: Wash Dark Colour Separately]""","""Tonyboy Brand Presents The Combo shirt of Yar...",shirt fit worn casual formal occasion pair jean,"Shoes & Sneakers For A Cool Smart Look.""",2677.0,984.251967


In [135]:
df_train['BULLET_POINTS'] = df_train['BULLET_POINTS'].astype(str)
print(df_train['BULLET_POINTS'].apply(lambda x: len(x.split(' '))).sum())

452222


In [136]:
import string
# Create a function to remove punctuations
def remove_punctuation(sentence: str) -> str:
    return sentence.translate(str.maketrans('', '', string.punctuation))

In [137]:
from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()
df_train['BULLET_POINTS'] = df_train['BULLET_POINTS'].apply(porter.stem)
df_train['BULLET_POINTS'] = df_train['BULLET_POINTS'].apply(remove_punctuation)

In [138]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
# CountVectorizer - name & categories
cv = CountVectorizer(min_df=10)
X_name = cv.fit_transform(df_train['BULLET_POINTS'])

In [139]:
from collections import Counter
def reducecolumns(Col):
    # remove columns with <= 1% non-zero entries (arbitrary choice so that notebook doesn't crash)
    n_docs = Counter(Col.nonzero()[1])
    cols_to_keep = [k for k, v in n_docs.items() if v > .0005 * Col.shape[0]]
    return Col[:, cols_to_keep]

In [140]:
print("Item Name Shape: " + str(reducecolumns(X_name).shape))

Item Name Shape: (59515, 2160)


In [141]:
X_brand1 = lb.fit_transform(df_train['BULLET_POINTS'])
X_description1 = tv.fit_transform(df_train['BULLET_POINTS'])

In [142]:
tfidf_weights = dict(zip(tv.get_feature_names_out(), tv.idf_))
tfidf_weights = pd.DataFrame(columns = ['BULLET_POINTS']).from_dict(dict(tfidf_weights), orient = 'index')
tfidf_weights.columns = ['BULLET_POINTS']

#Lowest tfidf scores -  Highest frequency words that get most importance:
tfidf_weights.sort_values(by=['BULLET_POINTS'], ascending=True).head(10)

,BULLET_POINTS
design,4.100428
quality,4.225467
easy,4.392598
product,4.398613
high,4.432879
fit,4.440714
perfect,4.443339
color,4.449668
material,4.499570
size,4.505707


<b> DESCRIPTION </b>

In [143]:
def cleanText(words):
    """The function to clean text"""
    words = re.sub("[^a-zA-Z]"," ",words)
    text = words.lower().split()
    return " ".join(text)

df_train['DESCRIPTION'] = df_train['DESCRIPTION'].apply(cleanText)

In [144]:
def remove_stopwords(text):
    """The function to removing stopwords"""
    text = [word.lower() for word in text.split() if word.lower() not in stop]
    return " ".join(text)

In [145]:
def word_lem(text):
    """The function to apply lemmatizing"""
    lem_text = [lem.lemmatize(word) for word in text.split()]
    return " ".join(lem_text)
df_train['DESCRIPTION'] = df_train['DESCRIPTION'].apply(remove_stopwords)
df_train['DESCRIPTION'] = df_train['DESCRIPTION'].apply(word_lem)

In [147]:
df_train.head()

,,,,,,PRODUCT_ID,TITLE,BULLET_POINTS,DESCRIPTION,PRODUCT_TYPE_ID,PRODUCT_LENGTH
1329311,"""Amscan 354001.54 Jumbo Margarita Glass",30 oz,"Caribbean Blue""","""[30 oz.",High quality paper,Drink in style with this plastic margarita glass,Perfect for parties,celebration add classy simple accent table set,serve cool drink oz jumbo plastic margarita gl...,1419.0,950.0
798322,"""K-Swiss 201 Classic Tennis Shoe (Infant/Toddler)",Black/Black,"5 M US Toddler""","""[Sleek low-top sneaker with vertically stitched toe",D-ring hardware lacing,"Perforated tongue]""","""K-Swiss 201 Classic Tennis Shoe (Infant/Toddler)",black black,u toddler,2449.0,750.0
1629663,"""Liakada Girls Argyle Supportive Sports Bra with Scoop Neckline Dance",Gym,Yoga,"Cheer! Black""","""[This sports bra top features contract criss-cross straps on the back and a scoop neckline for a clean finish.",Integrated bra shelf liner,Adjustable straps in a woven pattern,nylon spandex,sport bra top feature contract criss cross str...,2640.0,800.0
1833730,Khadims Men Black Slip-On Shoe - UK 9,"""[Upper Material: Synthetic / Sole Material: Airmax",Occasion: Formal / Utility: Office Formals,Closure: Slip-on / Toe Style: Closed,Shoe Style: Slip-On / Shoe Type: Shoe,Care Instructions: 1. Use a soft shoe brush to...,"""Khadim’s combines comfort and durability with...",pair khadim foot give extra confidence take gr...,khadim give unmatched value money experience k...,3246.0,1279.527558
2656446,TONYBOY Boy's Yarn Dyed Checks Shirt & Solid Shirt - Green/Fluo Orange (Pack of 2),"""[Fit Type: Regular Fit",Material: Cotton Fabric,Pattern : 1pcs. Y/D Checks Shirt || 1 pc. Solid Shirt,Sleeve: Full Sleeve || Neck: Regular Collar,"Wash Care: Wash Dark Colour Separately]""","""Tonyboy Brand Presents The Combo shirt of Yar...",shirt fit worn casual formal occasion pair jean,shoe sneaker cool smart look,2677.0,984.251967


In [148]:
df_train['DESCRIPTION'] = df_train['DESCRIPTION'].astype(str)
print(df_train['DESCRIPTION'].apply(lambda x: len(x.split(' '))).sum())

574446


In [149]:
import string
# Create a function to remove punctuations
def remove_punctuation(sentence: str) -> str:
    return sentence.translate(str.maketrans('', '', string.punctuation))

In [150]:
from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()
df_train['DECSRIPTION'] = df_train['DESCRIPTION'].apply(porter.stem)
df_train['DESCRIPTION'] = df_train['DESCRIPTION'].apply(remove_punctuation)

In [151]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
# CountVectorizer - name & categories
cv = CountVectorizer(min_df=10)
X_name2 = cv.fit_transform(df_train['DESCRIPTION'])

In [152]:
from collections import Counter
def reducecolumns(Col):
    # remove columns with <= 1% non-zero entries (arbitrary choice so that notebook doesn't crash)
    n_docs = Counter(Col.nonzero()[1])
    cols_to_keep = [k for k, v in n_docs.items() if v > .0005 * Col.shape[0]]
    return Col[:, cols_to_keep]

In [153]:
print("Item Name Shape: " + str(reducecolumns(X_name2).shape))

Item Name Shape: (59515, 2389)


In [154]:
X_brand2 = lb.fit_transform(df_train['DESCRIPTION'])
X_description2 = tv.fit_transform(df_train['DESCRIPTION'])

In [155]:
tfidf_weights = dict(zip(tv.get_feature_names_out(), tv.idf_))
tfidf_weights = pd.DataFrame(columns = ['DESCRIPTION']).from_dict(dict(tfidf_weights), orient = 'index')
tfidf_weights.columns = ['DESCRIPTION']

#Lowest tfidf scores -  Highest frequency words that get most importance:
tfidf_weights.sort_values(by=['DESCRIPTION'], ascending=True).head(10)

,DESCRIPTION
design,3.922470
quality,3.998357
cover,4.102296
material,4.109048
product,4.114709
easy,4.164768
color,4.183242
size,4.252033
fit,4.280216
perfect,4.288288


<b> TITLE </b>

In [158]:
def cleanText(words):
    """The function to clean text"""
    words = re.sub("[^a-zA-Z]"," ",words)
    text = words.lower().split()
    return " ".join(text)

df_train['TITLE'] = df_train['TITLE'].apply(cleanText)

In [157]:
def remove_stopwords(text):
    """The function to removing stopwords"""
    text = [word.lower() for word in text.split() if word.lower() not in stop]
    return " ".join(text)

In [159]:
def word_lem(text):
    """The function to apply lemmatizing"""
    lem_text = [lem.lemmatize(word) for word in text.split()]
    return " ".join(lem_text)
df_train['TITLE'] = df_train['TITLE'].apply(remove_stopwords)
df_train['TITLE'] = df_train['TITLE'].apply(word_lem)

In [162]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
# CountVectorizer - name & categories
cv = CountVectorizer(min_df=10)
X_name3 = cv.fit_transform(df_train['TITLE'])

In [161]:
from collections import Counter
def reducecolumns(Col):
    # remove columns with <= 1% non-zero entries (arbitrary choice so that notebook doesn't crash)
    n_docs = Counter(Col.nonzero()[1])
    cols_to_keep = [k for k, v in n_docs.items() if v > .0005 * Col.shape[0]]
    return Col[:, cols_to_keep]

In [163]:
print("Item Name Shape: " + str(reducecolumns(X_name3).shape))

Item Name Shape: (59515, 1982)


In [164]:
X_brand3 = lb.fit_transform(df_train['TITLE'])
X_description3 = tv.fit_transform(df_train['TITLE'])

In [165]:
tfidf_weights = dict(zip(tv.get_feature_names_out(), tv.idf_))
tfidf_weights = pd.DataFrame(columns = ['TITLE']).from_dict(dict(tfidf_weights), orient = 'index')
tfidf_weights.columns = ['TITLE']

#Lowest tfidf scores -  Highest frequency words that get most importance:
tfidf_weights.sort_values(by=['TITLE'], ascending=True).head(10)

,TITLE
easy,3.938525
design,4.103418
quality,4.260755
material,4.343356
size,4.443865
color,4.497903
product,4.508509
high,4.509070
fit,4.538702
perfect,4.572225


In [166]:
X_dummies = csr_matrix(pd.get_dummies(df_train[['TITLE', 'BULLET_POINTS', 'DESCRIPTION']], sparse=True).values)

In [167]:
from scipy.sparse import csr_matrix, hstack

In [168]:
# Combine everything together using Sparse matrix
sparse_merge = hstack((X_dummies, reducecolumns(X_brand1), reducecolumns(X_description1), reducecolumns(X_brand2), reducecolumns(X_description2), reducecolumns(X_brand3), reducecolumns(X_description3))).tocsr()

print(sparse_merge.shape)

(59515, 127266)


In [172]:
df_train['PRODUCT_LENGTH'] = pd.to_numeric(df_train['PRODUCT_LENGTH'], errors='coerce')
target = np.log(df_train['PRODUCT_LENGTH'].fillna(0) + 1)

In [173]:
print(target.shape)

(59515,)


In [170]:
from collections import Counter

n_docs = Counter(sparse_merge.nonzero()[1])
cols_to_keep = [k for k, v in n_docs.items() if v > .0005 * sparse_merge.shape[0]]
sparse_merge_filtered = sparse_merge[:, cols_to_keep]

In [174]:
#Split into train and test sets
from sklearn.model_selection import train_test_split
features_train, features_test, target_train, target_test = train_test_split(sparse_merge, target, test_size = 0.1, random_state = 1)

In [175]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [176]:
import keras

In [58]:
!pip install scikeras

In [177]:
from scikeras.wrappers import KerasRegressor

In [60]:
!pip install transformers

In [178]:
from sklearn.pipeline import Pipeline

In [182]:
def regressor_model():
    model = Sequential()
    model.add(Dense(units=7, kernel_initializer='uniform', activation='relu', input_dim=127266))
    model.add(Dense(1, kernel_initializer='uniform'))
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mae'])
    return model

In [183]:
features_train_processed = features_train.toarray()
estimators = []
estimators.append(('mlp', KerasRegressor(build_fn=regressor_model, verbose=1)))
pipeline = Pipeline(estimators)

In [184]:
pipeline.fit(features_train, target_train)

1674/1674 [==============================] - 25s 14ms/step - loss: 7.4030 - mae: 1.9082


Pipeline(steps=[('mlp',
                 KerasRegressor(build_fn=<function regressor_model at 0x7c025151cca0>))])

In [185]:
feature_test_modified=features_test.astype(np.float32).toarray()

In [186]:
target_pred = pipeline.predict(feature_test_modified)

186/186 [==============================] - 3s 17ms/step


In [187]:
from sklearn.metrics import mean_squared_error
import numpy as np

In [188]:
rmse = np.sqrt(mean_squared_error(target_test, target_pred))

print("Root Mean Square Error:", rmse)

RMSE: 2.5359473695910486


In [189]:
smape = np.mean(2.0 * np.abs(target_pred - target_test) / (np.abs(target_pred) + np.abs(target_test))) * 100.0

print(" Symmetric Mean Absolute Percentage Error:", smape)

SMAPE: 176.44262610760427
